# Websocket parser

## Dependencies

In [ ]:
from dataclasses import asdict, dataclass, field
from enum import Enum
import json
import re
from typing import Any, Dict, List, Optional

# Add Python lib to notebook
import os
import sys
module_path = os.path.abspath(os.path.join('..', 'lts-pipeline', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Add custom modules
from ltspipe.data.actions import Action, ActionType
from ltspipe.data.competitions import (
    DiffLap,
    InitialData,
    LengthUnit,
    Participant,
)
from ltspipe.data.enum import (
    CompetitionStage,
    CompetitionStatus,
    ParserSettings,
)
from ltspipe.messages import Message
from ltspipe.parsers.base import Parser

In [ ]:
str_msg = '{"competition_code": "los-santos-26-03-2023", "data": "init|p|\\nbest|hide|\\ncss|no3|border-bottom-color:#80FF00 !important; color:#000000 !important;\\ncss|no2|border-bottom-color:#FFFF00 !important; color:#000000 !important;\\ncss|no5|border-bottom-color:#FF8040 !important; color:#000000 !important;\\ncss|no1|border-bottom-color:#FFFF00 !important; color:#000000 !important;\\ncss|no4|border-bottom-color:#FF0080 !important; color:#FFFFFF !important;\\ncss|pena1|color:#f00; font-size:11px; font-weight: bold; text-align:center;\\neffects||Mostrar Efectos\\ncomments||Comentarios\\ntitle1||RESISTENCIA 3H\\ntitle2||CRONO\\ndyn1|text|00:20:00\\nlight|lr|\\nwth1||\\nwth2||\\nwth3||\\ntrack||LOS SANTOS (1100m)\\ncom||\\ngrid||<tbody><tr data-id=\\"r0\\" class=\\"head\\" data-pos=\\"0\\"><td data-id=\\"c1\\" data-type=\\"grp\\" data-pr=\\"6\\"></td><td data-id=\\"c2\\" data-type=\\"sta\\" data-pr=\\"1\\"></td><td data-id=\\"c3\\" data-type=\\"rk\\" data-pr=\\"1\\">Clasif.</td><td data-id=\\"c4\\" data-type=\\"no\\" data-pr=\\"1\\">Kart</td><td data-id=\\"c5\\" data-type=\\"dr\\" data-pr=\\"1\\" data-width=\\"23\\" data-min=\\"16\\">Equipo</td><td data-id=\\"c6\\" data-type=\\"llp\\" data-pr=\\"2\\" data-width=\\"14\\" data-min=\\"7\\">\\u00daltima vuelta</td><td data-id=\\"c7\\" data-type=\\"blp\\" data-pr=\\"1\\" data-width=\\"14\\" data-min=\\"7\\">Mejor vuelta</td><td data-id=\\"c8\\" data-type=\\"gap\\" data-pr=\\"4\\" data-width=\\"10\\" data-min=\\"7\\">Gap</td><td data-id=\\"c9\\" data-type=\\"int\\" data-pr=\\"5\\" data-width=\\"11\\" data-min=\\"7\\">Interv.</td><td data-id=\\"c10\\" data-type=\\"\\" data-pr=\\"6\\" data-width=\\"8\\" data-min=\\"6\\">Vueltas</td><td data-id=\\"c11\\" data-type=\\"otr\\" data-pr=\\"2\\" data-width=\\"10\\" data-min=\\"6\\">Tiempo en PIT</td><td data-id=\\"c12\\" data-type=\\"pit\\" data-pr=\\"2\\" data-width=\\"7\\" data-min=\\"7\\">Pits</td></tr><tr data-id=\\"r5625\\" data-pos=\\"1\\"><td data-id=\\"r5625c1\\" class=\\"in\\"></td><td data-id=\\"r5625c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5625c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5625c4\\" class=\\"no2\\">1</div></td><td data-id=\\"r5625c5\\" class=\\"dr\\">CREATIA RACING TEAM</td><td data-id=\\"r5625c6\\" class=\\"\\"></td><td data-id=\\"r5625c7\\" class=\\"ib\\"></td><td data-id=\\"r5625c8\\" class=\\"in\\"></td><td data-id=\\"r5625c9\\" class=\\"in\\"></td><td data-id=\\"r5625c10\\" class=\\"in\\"></td><td data-id=\\"r5625c11\\" class=\\"in\\"></td><td data-id=\\"r5625c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5626\\" data-pos=\\"2\\"><td data-id=\\"r5626c1\\" class=\\"in\\"></td><td data-id=\\"r5626c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5626c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5626c4\\" class=\\"no2\\">2</div></td><td data-id=\\"r5626c5\\" class=\\"dr\\">CREATIA KARTING TEAM</td><td data-id=\\"r5626c6\\" class=\\"\\"></td><td data-id=\\"r5626c7\\" class=\\"ib\\"></td><td data-id=\\"r5626c8\\" class=\\"in\\"></td><td data-id=\\"r5626c9\\" class=\\"in\\"></td><td data-id=\\"r5626c10\\" class=\\"in\\"></td><td data-id=\\"r5626c11\\" class=\\"in\\"></td><td data-id=\\"r5626c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5627\\" data-pos=\\"3\\"><td data-id=\\"r5627c1\\" class=\\"in\\"></td><td data-id=\\"r5627c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5627c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5627c4\\" class=\\"no2\\">3</div></td><td data-id=\\"r5627c5\\" class=\\"dr\\">CREATIA ENDURANCE TEAM</td><td data-id=\\"r5627c6\\" class=\\"\\"></td><td data-id=\\"r5627c7\\" class=\\"ib\\"></td><td data-id=\\"r5627c8\\" class=\\"in\\"></td><td data-id=\\"r5627c9\\" class=\\"in\\"></td><td data-id=\\"r5627c10\\" class=\\"in\\"></td><td data-id=\\"r5627c11\\" class=\\"in\\"></td><td data-id=\\"r5627c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5628\\" data-pos=\\"4\\"><td data-id=\\"r5628c1\\" class=\\"in\\"></td><td data-id=\\"r5628c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5628c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5628c4\\" class=\\"no2\\">4</div></td><td data-id=\\"r5628c5\\" class=\\"dr\\">PINK ARIZA CIRCUIT & ESCUELADEKART.COM</td><td data-id=\\"r5628c6\\" class=\\"\\"></td><td data-id=\\"r5628c7\\" class=\\"ib\\"></td><td data-id=\\"r5628c8\\" class=\\"in\\"></td><td data-id=\\"r5628c9\\" class=\\"in\\"></td><td data-id=\\"r5628c10\\" class=\\"in\\"></td><td data-id=\\"r5628c11\\" class=\\"in\\"></td><td data-id=\\"r5628c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5629\\" data-pos=\\"5\\"><td data-id=\\"r5629c1\\" class=\\"in\\"></td><td data-id=\\"r5629c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5629c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5629c4\\" class=\\"no2\\">5</div></td><td data-id=\\"r5629c5\\" class=\\"dr\\">CLUB KARTING MADRID</td><td data-id=\\"r5629c6\\" class=\\"\\"></td><td data-id=\\"r5629c7\\" class=\\"ib\\"></td><td data-id=\\"r5629c8\\" class=\\"in\\"></td><td data-id=\\"r5629c9\\" class=\\"in\\"></td><td data-id=\\"r5629c10\\" class=\\"in\\"></td><td data-id=\\"r5629c11\\" class=\\"in\\"></td><td data-id=\\"r5629c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5630\\" data-pos=\\"6\\"><td data-id=\\"r5630c1\\" class=\\"in\\"></td><td data-id=\\"r5630c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5630c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5630c4\\" class=\\"no2\\">6</div></td><td data-id=\\"r5630c5\\" class=\\"dr\\">CKM MOTORSPORT</td><td data-id=\\"r5630c6\\" class=\\"\\"></td><td data-id=\\"r5630c7\\" class=\\"ib\\"></td><td data-id=\\"r5630c8\\" class=\\"in\\"></td><td data-id=\\"r5630c9\\" class=\\"in\\"></td><td data-id=\\"r5630c10\\" class=\\"in\\"></td><td data-id=\\"r5630c11\\" class=\\"in\\"></td><td data-id=\\"r5630c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5631\\" data-pos=\\"7\\"><td data-id=\\"r5631c1\\" class=\\"in\\"></td><td data-id=\\"r5631c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5631c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5631c4\\" class=\\"no2\\">7</div></td><td data-id=\\"r5631c5\\" class=\\"dr\\">CKM ENDURANCE</td><td data-id=\\"r5631c6\\" class=\\"\\"></td><td data-id=\\"r5631c7\\" class=\\"ib\\"></td><td data-id=\\"r5631c8\\" class=\\"in\\"></td><td data-id=\\"r5631c9\\" class=\\"in\\"></td><td data-id=\\"r5631c10\\" class=\\"in\\"></td><td data-id=\\"r5631c11\\" class=\\"in\\"></td><td data-id=\\"r5631c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5632\\" data-pos=\\"8\\"><td data-id=\\"r5632c1\\" class=\\"in\\"></td><td data-id=\\"r5632c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5632c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5632c4\\" class=\\"no2\\">8</div></td><td data-id=\\"r5632c5\\" class=\\"dr\\">CKM SPORT TEAM</td><td data-id=\\"r5632c6\\" class=\\"\\"></td><td data-id=\\"r5632c7\\" class=\\"ib\\"></td><td data-id=\\"r5632c8\\" class=\\"in\\"></td><td data-id=\\"r5632c9\\" class=\\"in\\"></td><td data-id=\\"r5632c10\\" class=\\"in\\"></td><td data-id=\\"r5632c11\\" class=\\"in\\"></td><td data-id=\\"r5632c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5633\\" data-pos=\\"9\\"><td data-id=\\"r5633c1\\" class=\\"in\\"></td><td data-id=\\"r5633c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5633c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5633c4\\" class=\\"no2\\">9</div></td><td data-id=\\"r5633c5\\" class=\\"dr\\">HISPANO MALEMAN RACING TEAM</td><td data-id=\\"r5633c6\\" class=\\"\\"></td><td data-id=\\"r5633c7\\" class=\\"ib\\"></td><td data-id=\\"r5633c8\\" class=\\"in\\"></td><td data-id=\\"r5633c9\\" class=\\"in\\"></td><td data-id=\\"r5633c10\\" class=\\"in\\"></td><td data-id=\\"r5633c11\\" class=\\"in\\"></td><td data-id=\\"r5633c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5634\\" data-pos=\\"10\\"><td data-id=\\"r5634c1\\" class=\\"in\\"></td><td data-id=\\"r5634c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5634c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5634c4\\" class=\\"no2\\">10</div></td><td data-id=\\"r5634c5\\" class=\\"dr\\">HISPANO ALEMAN ELITE RACING TEAM</td><td data-id=\\"r5634c6\\" class=\\"\\"></td><td data-id=\\"r5634c7\\" class=\\"ib\\"></td><td data-id=\\"r5634c8\\" class=\\"in\\"></td><td data-id=\\"r5634c9\\" class=\\"in\\"></td><td data-id=\\"r5634c10\\" class=\\"in\\"></td><td data-id=\\"r5634c11\\" class=\\"in\\"></td><td data-id=\\"r5634c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5635\\" data-pos=\\"11\\"><td data-id=\\"r5635c1\\" class=\\"in\\"></td><td data-id=\\"r5635c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5635c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5635c4\\" class=\\"no2\\">11</div></td><td data-id=\\"r5635c5\\" class=\\"dr\\">HISPANO ALEMAN TALLER ESTRECHO</td><td data-id=\\"r5635c6\\" class=\\"\\"></td><td data-id=\\"r5635c7\\" class=\\"ib\\"></td><td data-id=\\"r5635c8\\" class=\\"in\\"></td><td data-id=\\"r5635c9\\" class=\\"in\\"></td><td data-id=\\"r5635c10\\" class=\\"in\\"></td><td data-id=\\"r5635c11\\" class=\\"in\\"></td><td data-id=\\"r5635c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5636\\" data-pos=\\"12\\"><td data-id=\\"r5636c1\\" class=\\"in\\"></td><td data-id=\\"r5636c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5636c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5636c4\\" class=\\"no2\\">12</div></td><td data-id=\\"r5636c5\\" class=\\"dr\\">HORMIGONES MV</td><td data-id=\\"r5636c6\\" class=\\"\\"></td><td data-id=\\"r5636c7\\" class=\\"ib\\"></td><td data-id=\\"r5636c8\\" class=\\"in\\"></td><td data-id=\\"r5636c9\\" class=\\"in\\"></td><td data-id=\\"r5636c10\\" class=\\"in\\"></td><td data-id=\\"r5636c11\\" class=\\"in\\"></td><td data-id=\\"r5636c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5637\\" data-pos=\\"13\\"><td data-id=\\"r5637c1\\" class=\\"in\\"></td><td data-id=\\"r5637c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5637c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5637c4\\" class=\\"no2\\">13</div></td><td data-id=\\"r5637c5\\" class=\\"dr\\">CONSTRUCCIONES MV</td><td data-id=\\"r5637c6\\" class=\\"\\"></td><td data-id=\\"r5637c7\\" class=\\"ib\\"></td><td data-id=\\"r5637c8\\" class=\\"in\\"></td><td data-id=\\"r5637c9\\" class=\\"in\\"></td><td data-id=\\"r5637c10\\" class=\\"in\\"></td><td data-id=\\"r5637c11\\" class=\\"in\\"></td><td data-id=\\"r5637c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5638\\" data-pos=\\"14\\"><td data-id=\\"r5638c1\\" class=\\"in\\"></td><td data-id=\\"r5638c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5638c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5638c4\\" class=\\"no2\\">14</div></td><td data-id=\\"r5638c5\\" class=\\"dr\\">MV INMOBILIARIO</td><td data-id=\\"r5638c6\\" class=\\"\\"></td><td data-id=\\"r5638c7\\" class=\\"ib\\"></td><td data-id=\\"r5638c8\\" class=\\"in\\"></td><td data-id=\\"r5638c9\\" class=\\"in\\"></td><td data-id=\\"r5638c10\\" class=\\"in\\"></td><td data-id=\\"r5638c11\\" class=\\"in\\"></td><td data-id=\\"r5638c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5639\\" data-pos=\\"15\\"><td data-id=\\"r5639c1\\" class=\\"in\\"></td><td data-id=\\"r5639c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5639c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5639c4\\" class=\\"no2\\">15</div></td><td data-id=\\"r5639c5\\" class=\\"dr\\">LMKC ENDURANCE</td><td data-id=\\"r5639c6\\" class=\\"\\"></td><td data-id=\\"r5639c7\\" class=\\"ib\\"></td><td data-id=\\"r5639c8\\" class=\\"in\\"></td><td data-id=\\"r5639c9\\" class=\\"in\\"></td><td data-id=\\"r5639c10\\" class=\\"in\\"></td><td data-id=\\"r5639c11\\" class=\\"in\\"></td><td data-id=\\"r5639c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5640\\" data-pos=\\"16\\"><td data-id=\\"r5640c1\\" class=\\"in\\"></td><td data-id=\\"r5640c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5640c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5640c4\\" class=\\"no2\\">16</div></td><td data-id=\\"r5640c5\\" class=\\"dr\\">JCRTEAM.COM</td><td data-id=\\"r5640c6\\" class=\\"\\"></td><td data-id=\\"r5640c7\\" class=\\"ib\\"></td><td data-id=\\"r5640c8\\" class=\\"in\\"></td><td data-id=\\"r5640c9\\" class=\\"in\\"></td><td data-id=\\"r5640c10\\" class=\\"in\\"></td><td data-id=\\"r5640c11\\" class=\\"in\\"></td><td data-id=\\"r5640c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5641\\" data-pos=\\"17\\"><td data-id=\\"r5641c1\\" class=\\"in\\"></td><td data-id=\\"r5641c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5641c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5641c4\\" class=\\"no2\\">17</div></td><td data-id=\\"r5641c5\\" class=\\"dr\\">PROJECT EQTOX</td><td data-id=\\"r5641c6\\" class=\\"\\"></td><td data-id=\\"r5641c7\\" class=\\"ib\\"></td><td data-id=\\"r5641c8\\" class=\\"in\\"></td><td data-id=\\"r5641c9\\" class=\\"in\\"></td><td data-id=\\"r5641c10\\" class=\\"in\\"></td><td data-id=\\"r5641c11\\" class=\\"in\\"></td><td data-id=\\"r5641c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5642\\" data-pos=\\"18\\"><td data-id=\\"r5642c1\\" class=\\"in\\"></td><td data-id=\\"r5642c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5642c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5642c4\\" class=\\"no2\\">18</div></td><td data-id=\\"r5642c5\\" class=\\"dr\\">PROJECT DAYTON</td><td data-id=\\"r5642c6\\" class=\\"\\"></td><td data-id=\\"r5642c7\\" class=\\"ib\\"></td><td data-id=\\"r5642c8\\" class=\\"in\\"></td><td data-id=\\"r5642c9\\" class=\\"in\\"></td><td data-id=\\"r5642c10\\" class=\\"in\\"></td><td data-id=\\"r5642c11\\" class=\\"in\\"></td><td data-id=\\"r5642c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5643\\" data-pos=\\"19\\"><td data-id=\\"r5643c1\\" class=\\"in\\"></td><td data-id=\\"r5643c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5643c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5643c4\\" class=\\"no2\\">19</div></td><td data-id=\\"r5643c5\\" class=\\"dr\\">RADIUS JARAMAFAN</td><td data-id=\\"r5643c6\\" class=\\"\\"></td><td data-id=\\"r5643c7\\" class=\\"ib\\"></td><td data-id=\\"r5643c8\\" class=\\"in\\"></td><td data-id=\\"r5643c9\\" class=\\"in\\"></td><td data-id=\\"r5643c10\\" class=\\"in\\"></td><td data-id=\\"r5643c11\\" class=\\"in\\"></td><td data-id=\\"r5643c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5644\\" data-pos=\\"20\\"><td data-id=\\"r5644c1\\" class=\\"in\\"></td><td data-id=\\"r5644c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5644c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5644c4\\" class=\\"no2\\">20</div></td><td data-id=\\"r5644c5\\" class=\\"dr\\">JARAMAFAN RACING</td><td data-id=\\"r5644c6\\" class=\\"\\"></td><td data-id=\\"r5644c7\\" class=\\"ib\\"></td><td data-id=\\"r5644c8\\" class=\\"in\\"></td><td data-id=\\"r5644c9\\" class=\\"in\\"></td><td data-id=\\"r5644c10\\" class=\\"in\\"></td><td data-id=\\"r5644c11\\" class=\\"in\\"></td><td data-id=\\"r5644c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5645\\" data-pos=\\"21\\"><td data-id=\\"r5645c1\\" class=\\"in\\"></td><td data-id=\\"r5645c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5645c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5645c4\\" class=\\"no2\\">21</div></td><td data-id=\\"r5645c5\\" class=\\"dr\\">SPARTAN SPORT</td><td data-id=\\"r5645c6\\" class=\\"\\"></td><td data-id=\\"r5645c7\\" class=\\"ib\\"></td><td data-id=\\"r5645c8\\" class=\\"in\\"></td><td data-id=\\"r5645c9\\" class=\\"in\\"></td><td data-id=\\"r5645c10\\" class=\\"in\\"></td><td data-id=\\"r5645c11\\" class=\\"in\\"></td><td data-id=\\"r5645c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5646\\" data-pos=\\"22\\"><td data-id=\\"r5646c1\\" class=\\"in\\"></td><td data-id=\\"r5646c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5646c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5646c4\\" class=\\"no2\\">22</div></td><td data-id=\\"r5646c5\\" class=\\"dr\\">SPARTAN RT</td><td data-id=\\"r5646c6\\" class=\\"\\"></td><td data-id=\\"r5646c7\\" class=\\"ib\\"></td><td data-id=\\"r5646c8\\" class=\\"in\\"></td><td data-id=\\"r5646c9\\" class=\\"in\\"></td><td data-id=\\"r5646c10\\" class=\\"in\\"></td><td data-id=\\"r5646c11\\" class=\\"in\\"></td><td data-id=\\"r5646c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5647\\" data-pos=\\"23\\"><td data-id=\\"r5647c1\\" class=\\"in\\"></td><td data-id=\\"r5647c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5647c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5647c4\\" class=\\"no2\\">23</div></td><td data-id=\\"r5647c5\\" class=\\"dr\\">KTA</td><td data-id=\\"r5647c6\\" class=\\"\\"></td><td data-id=\\"r5647c7\\" class=\\"ib\\"></td><td data-id=\\"r5647c8\\" class=\\"in\\"></td><td data-id=\\"r5647c9\\" class=\\"in\\"></td><td data-id=\\"r5647c10\\" class=\\"in\\"></td><td data-id=\\"r5647c11\\" class=\\"in\\"></td><td data-id=\\"r5647c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5648\\" data-pos=\\"24\\"><td data-id=\\"r5648c1\\" class=\\"in\\"></td><td data-id=\\"r5648c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5648c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5648c4\\" class=\\"no2\\">24</div></td><td data-id=\\"r5648c5\\" class=\\"dr\\">CHETOS</td><td data-id=\\"r5648c6\\" class=\\"\\"></td><td data-id=\\"r5648c7\\" class=\\"ib\\"></td><td data-id=\\"r5648c8\\" class=\\"in\\"></td><td data-id=\\"r5648c9\\" class=\\"in\\"></td><td data-id=\\"r5648c10\\" class=\\"in\\"></td><td data-id=\\"r5648c11\\" class=\\"in\\"></td><td data-id=\\"r5648c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5649\\" data-pos=\\"25\\"><td data-id=\\"r5649c1\\" class=\\"in\\"></td><td data-id=\\"r5649c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5649c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5649c4\\" class=\\"no2\\">25</div></td><td data-id=\\"r5649c5\\" class=\\"dr\\">SENNADORES RACING</td><td data-id=\\"r5649c6\\" class=\\"\\"></td><td data-id=\\"r5649c7\\" class=\\"ib\\"></td><td data-id=\\"r5649c8\\" class=\\"in\\"></td><td data-id=\\"r5649c9\\" class=\\"in\\"></td><td data-id=\\"r5649c10\\" class=\\"in\\"></td><td data-id=\\"r5649c11\\" class=\\"in\\"></td><td data-id=\\"r5649c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5650\\" data-pos=\\"26\\"><td data-id=\\"r5650c1\\" class=\\"in\\"></td><td data-id=\\"r5650c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5650c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5650c4\\" class=\\"no2\\">26</div></td><td data-id=\\"r5650c5\\" class=\\"dr\\">SRT</td><td data-id=\\"r5650c6\\" class=\\"\\"></td><td data-id=\\"r5650c7\\" class=\\"ib\\"></td><td data-id=\\"r5650c8\\" class=\\"in\\"></td><td data-id=\\"r5650c9\\" class=\\"in\\"></td><td data-id=\\"r5650c10\\" class=\\"in\\"></td><td data-id=\\"r5650c11\\" class=\\"in\\"></td><td data-id=\\"r5650c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5651\\" data-pos=\\"27\\"><td data-id=\\"r5651c1\\" class=\\"in\\"></td><td data-id=\\"r5651c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5651c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5651c4\\" class=\\"no2\\">27</div></td><td data-id=\\"r5651c5\\" class=\\"dr\\">CORCHINETA RACING</td><td data-id=\\"r5651c6\\" class=\\"\\"></td><td data-id=\\"r5651c7\\" class=\\"ib\\"></td><td data-id=\\"r5651c8\\" class=\\"in\\"></td><td data-id=\\"r5651c9\\" class=\\"in\\"></td><td data-id=\\"r5651c10\\" class=\\"in\\"></td><td data-id=\\"r5651c11\\" class=\\"in\\"></td><td data-id=\\"r5651c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5652\\" data-pos=\\"28\\"><td data-id=\\"r5652c1\\" class=\\"in\\"></td><td data-id=\\"r5652c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5652c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5652c4\\" class=\\"no2\\">28</div></td><td data-id=\\"r5652c5\\" class=\\"dr\\">MILLE MIGLIA</td><td data-id=\\"r5652c6\\" class=\\"\\"></td><td data-id=\\"r5652c7\\" class=\\"ib\\"></td><td data-id=\\"r5652c8\\" class=\\"in\\"></td><td data-id=\\"r5652c9\\" class=\\"in\\"></td><td data-id=\\"r5652c10\\" class=\\"in\\"></td><td data-id=\\"r5652c11\\" class=\\"in\\"></td><td data-id=\\"r5652c12\\" class=\\"in\\"></td></tr><tr data-id=\\"r5653\\" data-pos=\\"29\\"><td data-id=\\"r5653c1\\" class=\\"in\\"></td><td data-id=\\"r5653c2\\" class=\\"in\\"></td><td class=\\"rk\\"><div><p data-id=\\"r5653c3\\" class=\\"\\"></p></div></td><td class=\\"no\\"><div data-id=\\"r5653c4\\" class=\\"no2\\">29</div></td><td data-id=\\"r5653c5\\" class=\\"dr\\">MRM RACING TEAM</td><td data-id=\\"r5653c6\\" class=\\"\\"></td><td data-id=\\"r5653c7\\" class=\\"ib\\"></td><td data-id=\\"r5653c8\\" class=\\"in\\"></td><td data-id=\\"r5653c9\\" class=\\"in\\"></td><td data-id=\\"r5653c10\\" class=\\"in\\"></td><td data-id=\\"r5653c11\\" class=\\"in\\"></td><td data-id=\\"r5653c12\\" class=\\"in\\"></td></tr></tbody>\\nmsg||", "source": "ws-listener", "created_at": 1679807829.29123, "updated_at": 1679807829.364932}'

In [ ]:
msg = Message.decode(str_msg)

In [ ]:
print(msg.get_data())

## Parser: init messages

In [ ]:
class WsInitParser(Parser):
    """
    Parse the initializer data from a websocket.
    """

    FILTER_STAGES = {
        'CRONO': CompetitionStage.QUALIFYING,
        'QUALIFYING': CompetitionStage.QUALIFYING,
        'QUALY': CompetitionStage.QUALIFYING,
        'QUALI': CompetitionStage.QUALIFYING,
        'RACE': CompetitionStage.RACE,
        'CARRERA': CompetitionStage.RACE,
        'RESISTENCIA': CompetitionStage.RACE,
        'ENDURANCE': CompetitionStage.RACE,
    }

    def __init__(self) -> None:
        """Construct."""
        self._filters = {
            'by_id': {
                'rk': ParserSettings.TIMING_RANKING,
                'no': ParserSettings.TIMING_KART_NUMBER,
                'dr': ParserSettings.TIMING_NAME,
                'llp': ParserSettings.TIMING_LAST_LAP_TIME,
                'blp': ParserSettings.TIMING_BEST_TIME,
                'gap': ParserSettings.TIMING_GAP,
                'int': ParserSettings.TIMING_INTERVAL,
                'pit': ParserSettings.TIMING_PITS,
            },
            'by_name': {
                'equipo': ParserSettings.TIMING_NAME,
                'kart': ParserSettings.TIMING_KART_NUMBER,
                '\\u00daltima vuelta': ParserSettings.TIMING_LAST_LAP_TIME,
                'mejor vuelta': ParserSettings.TIMING_BEST_TIME,
                'gap': ParserSettings.TIMING_GAP,
                'interv.': ParserSettings.TIMING_INTERVAL,
                'intervalo': ParserSettings.TIMING_INTERVAL,
                'vueltas': ParserSettings.TIMING_LAPS,
                'tiempo en pit': ParserSettings.TIMING_PIT_TIME,
                'pits': ParserSettings.TIMING_PITS,
            },
        }

    def parse(self, msg: Message) -> List[Action]:
        """
        Analyse and/or parse a given message data.

        Params:
            msg (Any): Message to parse.

        Returns:
            List[Action]: list of actions and their respective parsed data.
        """
        data = msg.get_data()
        if self._is_initializer_data(data):
            parsed_data = self._parse_init_data(data)
            action = Action(type=ActionType.INITIALIZE, data=parsed_data)
            return [action]
        return []

    def _is_initializer_data(self, data: Any) -> bool:
        """Check if it is an initializer data."""
        return (isinstance(data, str)
                and re.match(r'^init\|p\|', data) is not None)

    def _parse_init_data(self, data: str) -> dict:
        """Parse content in the raw data."""
        raw_parts = re.split(r'\n+', data, flags=re.MULTILINE)
        raw_parts = [re.split(r'\|', p, flags=re.MULTILINE) for p in raw_parts]
        parts = {p[0]:p for p in raw_parts}

        stage = self._parse_stage(parts['title2'][2])
        remaining_length = self._parse_length(parts['dyn1'][2])
        status = (CompetitionStatus.ONGOING if remaining_length.value == 0
                  else CompetitionStatus.FINISHED)

        # Parse intial content
        raw_content = parts['grid'][2]
        initial_rows = re.findall(r'<tr[^>]*>.+?</tr>', raw_content, flags=re.S)

        headers = self._parse_headers(initial_rows[0])
        participants = self._parse_participants(headers, initial_rows[1:])

        parsers_settings = headers
        initial_data = InitialData(
            reference_time=0,
            reference_current_offset=0,
            stage=stage,
            status=status,
            remaining_length=remaining_length,
            participants=participants,
            parsers_settings=headers,
        )
        return initial_data.dict()

    def _parse_stage(self, raw: str) -> str:
        """Parse competition stage."""
        if raw in self.FILTER_STAGES:
            return self.FILTER_STAGES[raw]
        raise Exception(f'Unknown stage: {raw}')

    def _parse_length(self, raw: str) -> str:
        """Parse remaining length of the competition."""
        return DiffLap(
            value=0, # self._time_to_millis(raw),
            unit=LengthUnit.MILLIS,
        )

    def _parse_headers(self, first_row: str) -> Dict[str, str]:
        """Parse headers from the first row."""
        header_data = {}
        items = re.findall(
            r'<td[^>]*data-type="([^"]+)"[^>]*>(.*?)</td>',
            first_row,
            flags=re.S)
        for i, item in enumerate(items):
            id_match = self.__get_by_key(item[0], self._filters['by_id'])
            name_match = self.__get_by_key(item[1], self._filters['by_name'])

            if id_match is None and name_match is None:
                continue
            if id_match is not None and name_match is None:
                header_data[id_match] = f'c{i + 1}'
            elif id_match is None and name_match is not None:
                header_data[name_match] = f'c{i + 1}'
            elif (id_match is not None
                    and name_match is not None
                    and id_match == name_match):
                header_data[id_match] = f'c{i + 1}'
            else:
                raise Exception(f'Cannot parse column {i + 1} of headers '
                                f'({id_match} != {name_match}).')

        return header_data

    def __get_by_key(
            self,
            value: str,
            filters: Dict[str, str]) -> Optional[str]:
        """Get by key."""
        value = value.lower()
        if value in filters:
            return filters[value]
        return None

    def _parse_participants(
            self,
            headers: Dict[str, str],
            rows: List[str]) -> Dict[str, Participant]:
        """Parse participants details."""
        participants = {}
        for row in rows:
            items = re.findall(
                r'<td.+?data-id="([^"]+)(c\d+)"[^>]*>(.*?)</td>',
                row,
                flags=re.S)
            fields = {}
            for item in items:
                if item[1] not in headers:
                    continue
                field_name = headers[item[1]]
                field_value = self._remove_html_tags(item[2])
                fields[field_name] = None if field_value == '' else field_value

            match = re.search(r'<tr[^>]*data-id="([^"]+)"', row, flags=re.S)
            if match is None:
                raise Exception(f'Could not parse code in: {row}')
            participant_code = match[1]

            match = re.search(r'<tr[^>]*data-pos="([^"]+)"', row, flags=re.S)
            ranking = None if match is None else match[1]

            p = Participant(
                participant_code=participant_code,
                ranking=self._cast_number(ranking),
                kart_number=self._cast_number(fields.get('KART_NUMBER', None)),
                team_name=fields.get('NAME', None),
                driver_name=None,
                last_lap_time=self._time_to_millis(
                    fields.get('LAST_LAP_TIME', None)),
                best_time=self._time_to_millis(fields.get('BEST_TIME', None)),
                gap=self._parse_diff_lap(fields.get('GAP', None)),
                interval=self._parse_diff_lap(fields.get('INTERVAL', None)),
                laps=self._cast_number(fields.get('LAPS', None)),
                pits=self._cast_number(fields.get('PITS', None)),
                pit_time=self._time_to_millis(fields.get('PIT_TIME', None)),
            )
            participants[participant_code] = p

        return participants

    def _remove_html_tags(self, text: str) -> str:
        """Remove HTML tags from a string."""
        return re.sub('<[^>]+>', '', text)

    def _time_to_millis(self, lap_time: Optional[str]) -> Optional[int]:
        """Transform a lap time into milliseconds."""
        if lap_time is None:
            return None
        lap_time = lap_time.strip()
        match = re.search(r'^\+?(?:(\d+):)?(\d+)\.(\d+)?$', lap_time)
        if match is None:
            return None
        else:
            parts = [int(p) if p else 0 for p in match.groups()]
            return parts[0] * 60000 + parts[1] * 1000 + parts[2]

    def _parse_diff_lap(self, diff_lap: Optional[str]) -> Optional[DiffLap]:
        """Parse the difference between laps."""
        if diff_lap is None:
            return None
        diff_lap = diff_lap.strip()
        match_laps = re.search(
            r'^\+?(\d+) (?:vueltas?|laps?)$', diff_lap.lower())
        if match_laps:
            return DiffLap(
                value=int(match_laps[1]),
                unit=LengthUnit.LAPS,
            )

        diff_value = self._time_to_millis(diff_lap)
        if diff_value is not None:
            return DiffLap(
                value=diff_value,
                unit=LengthUnit.MILLIS,
            )

        return None

    def _cast_number(self, value: Optional[str]) -> Optional[int]:
        """Cast a string to int."""
        return None if value is None else int(value)


In [ ]:
# Sample
parser = WsInitParser()
new_msg = parser(msg)

print(new_msg)

In [ ]:
lap_time = '2:12.283'
lap_time = '00:20:00'
lap_time = '54.'

match = re.search(r'^\+?(?:(?:(\d+):)?(\d+):)?(\d+)(?:\.(\d+)?)?$', lap_time)
match.groups()